In [1]:
import requests
import time
import json
from tqdm import tqdm, trange
from itertools import chain

In [2]:
food_schema = ('id', 'name', 'seo_name', 'img')
winery_schema = ('id', 'name', 'seo_name')
grapes_schema = ('id', 'name', 'seo_name')
wine_schema = ('id', 'name', 'seo_name', 'style', 'region', 'winery', 'type_id', 'ratings_sum', 'ratings_count')
vintage_schema = ('id', 'name', 'seo_name', 'year', 'wine', 'img', 'ratings_sum', 'ratings_count')
style_schema = ('id', 'name')
region_schema = ('id', 'name', 'seo_name', 'country')
country_schema = ('code', 'name')
# style_schema = ('id')
style_food_pairings_schema = ('style_id', 'food_id')
grapes_pairing_schema = ('style_id', 'grape_id')
# regions = ('id', 'country', 'name_en', 'seo_name', 'name', 'background_image')
user_schema = ('id', 'seo_name', 'alias')
review_schema = ('id', 'user_id', 'vintage_id', 'note', 'rating')

def parse_single_vintage(raw):
    
    vintage = raw['vintage']
    wine = vintage['wine']
    winery = wine['winery']
    style = wine['style']
    region = wine['region']
    country = region['country']
    
    # vintage
    vintages[vintage['id']] = {
        'name': vintage['name'],
        'seo_name': vintage['seo_name'],
        'year': vintage['year'],
        'wine': vintage['wine']['id'],
        'img': vintage['image']['variations']['small_square'],
    }
    # wine
    wines[wine['id']] = {
        'name': wine['name'],
        'seo_name': wine['seo_name'],
        'style': style['id'] if style else None,
        'region': region['id'],
        'winery': winery['id'],
        'type_id': wine['type_id']
    }
    # winery
    wineries[winery['id']] = {
        'name': winery['name'],
        'seo_name': winery['seo_name'],
    }


    
    # regions
    regions[region['id']] = {
        'name': region['name'],
        'seo_name': region['seo_name'],
        'country': country['code'],
    }
    
    # countries
    countries[country['code']] = {
        'name': country['name'],
    }
    
    #reviews
    revs = []
    for page in range(1,10):
        revs.append(
            json.loads(
                requests.get(
                    'https://www.vivino.com/api/wines/{}/reviews?year={}&page={}'.format(wine['id'], vintage['year'], page),
                    headers=http_headers,
                ).text
            )['reviews']
        )
    
    for review in chain(*revs):
        if review['vintage']['id'] in vintages:
            user = review['user']
            reviews[review['id']] = {
                'rating': review['rating'],
                'note': review['note'].replace('\n', ' ').replace('#', ''),
                'vintage_id': review['vintage']['id'],
                'user_id': user['id'],
            }
            users[user['id']] = {
                'seo_name': user['seo_name'],
                'alias': user['alias'],
            }
    

    if style:
        # grapes + pairing
        for grape in style['grapes']:
            grapes_pairings.add((style['id'], grape['id']))
            grapes[grape['id']] = {
                'name': grape['name'],
                'seo_name': grape['seo_name'],
            }
        # style
        styles[style['id']] = {
            'name': style['name'] if style else None,
        }
            
        # food + pairing
        for food in style['food']:
            style_food_pairings.add((style['id'], food['id']))
            foods[food['id']] = {
                'name': food['name'],
                'seo_name': food['seo_name'],
                'img': food['background_image']['variations']['small'],
            }


In [4]:
http_headers = {
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',
'x-requested-with': 'XMLHttpRequest',
'content-type': 'application/json'
}
url = (
    'https://www.vivino.com/api/explore/explore?'
    'country_code=ru&'
    'currency_code=RUB&'
    'grape_filter=varietal&'
    'merchant_id=&'
    'min_rating=1&'
    'order_by=ratings_count&'
    'order=desc&'
    'page={}&'
    'price_range_max=12500&'
    'price_range_min=0'
)
foods = {}
wineries = {}
grapes = {}
wines = {}
vintages = {}
styles = {}
countries = {}
regions = {}
reviews = {}
users = {}
style_food_pairings = set()
grapes_pairings = set()
n_pages = 2
for i in trange(n_pages):
    time.sleep(1)
    r = requests.get(url.format(i),headers=http_headers)
    response = json.loads(r.text)
    for raw in tqdm(response['explore_vintage']['records']):
        parse_single_vintage(raw)



  0%|          | 0/2 [00:00<?, ?it/s]


  0%|          | 0/25 [00:00<?, ?it/s]


  4%|▍         | 1/25 [00:06<02:26,  6.11s/it]


  8%|▊         | 2/25 [00:11<02:12,  5.78s/it]


 12%|█▏        | 3/25 [00:16<02:03,  5.60s/it]


 16%|█▌        | 4/25 [00:23<02:08,  6.12s/it]


 20%|██        | 5/25 [00:29<02:02,  6.13s/it]


 24%|██▍       | 6/25 [00:36<01:58,  6.23s/it]


 28%|██▊       | 7/25 [00:42<01:53,  6.30s/it]


 32%|███▏      | 8/25 [00:48<01:46,  6.27s/it]


 36%|███▌      | 9/25 [00:55<01:40,  6.25s/it]


 40%|████      | 10/25 [00:59<01:23,  5.54s/it]


 44%|████▍     | 11/25 [01:03<01:14,  5.35s/it]


 48%|████▊     | 12/25 [01:10<01:14,  5.74s/it]


 52%|█████▏    | 13/25 [01:16<01:08,  5.72s/it]


 56%|█████▌    | 14/25 [01:22<01:05,  5.97s/it]


 60%|██████    | 15/25 [01:27<00:56,  5.66s/it]


 64%|██████▍   | 16/25 [01:36<00:58,  6.54s/it]


 68%|██████▊   | 17/25 [01:42<00:51,  6.38s/it]


 72%|███████▏  | 18/25 [01:47<00:42,  6.13s/it]


 76%|███████▌  | 19/25 [01

In [7]:
def make_formatter(schema):
    def csvy(args):
        k, v = args
        s = [str(k)]
        for col in schema[1:]:
            s.append(str(v[col]) if v.get(col, '') else '')
        return '#'.join(s)
    return csvy

In [8]:
with open('food.csv', 'w') as f:
    f.write('\n'.join(map(make_formatter(food_schema), foods.items())))
    
with open('style_food_pairings.csv', 'w') as f:
    f.write('\n'.join(map(lambda x: '#'.join(map(str, x)), style_food_pairings)))

with open('styles.csv', 'w') as f:
    f.write('\n'.join(map(make_formatter(style_schema), styles.items())))

with open('vintages.csv', 'w') as f:
    f.write('\n'.join(map(make_formatter(vintage_schema), vintages.items())))

with open('wines.csv', 'w') as f:
    f.write('\n'.join(map(make_formatter(wine_schema), wines.items())))

with open('regions.csv', 'w') as f:
    f.write('\n'.join(map(make_formatter(region_schema), regions.items())))

with open('countries.csv', 'w') as f:
    f.write('\n'.join(map(make_formatter(country_schema), countries.items())))

with open('grapes.csv', 'w') as f:
    f.write('\n'.join(map(make_formatter(grapes_schema), grapes.items())))

with open('wineries.csv', 'w') as f:
    f.write('\n'.join(map(make_formatter(winery_schema), wineries.items())))
    
with open('style_grapes_pairings.csv', 'w') as f:
    f.write('\n'.join(map(lambda x: '#'.join(map(str, x)), grapes_pairings)))
    
with open('users.csv', 'w') as f:
    f.write('\n'.join(map(make_formatter(user_schema), users.items())))

with open('reviews.csv', 'w') as f:
    f.write('\n'.join(map(make_formatter(review_schema), reviews.items())))

In [9]:
import os

TABLES = [
    'food', 'grapes', 'styles', 'wineries', 'countries', 'regions',
    'style_grapes_pairings', 'style_food_pairings', 'wines', 'vintages',
    'users', 'reviews'
]
SCRIPT_TEMPLATE = 'COPY {} FROM \'{}.csv\' DELIMITER \'#\' CSV;'
with open('fill.sql', 'w') as f:
    f.write(
        '\n'.join(
            map(
                lambda table_name : SCRIPT_TEMPLATE.format(table_name, os.path.abspath(table_name)),
                TABLES
            )
        )
    )